In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [ ]:
# 위노그라드 알고리즘 설정
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'

# 데이터 로드

In [ ]:
rootPath = './vision/'

In [ ]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [ ]:
imageGenerator = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True,
    preprocessing_function = get_random_eraser(v_l=0, v_h=255),
    validation_split=.1
)

In [ ]:
trainGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'training_set'),
    target_size=(100, 100),
    subset='training'
)

In [ ]:
validationGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'training_set'),
    target_size=(100, 100),
    subset='validation'
)

# 모델 구성

In [ ]:
model = Sequential()
model.add(ResNet50(include_top=True, weights=None, input_shape=(100, 100, 3), classes=3))

model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics=['acc'],
)

In [ ]:
epochs = 150
history = model.fit_generator(
    trainGen, 
    epochs=epochs,
    steps_per_epoch=trainGen.samples / epochs, 
    validation_data=validationGen,
    validation_steps=trainGen.samples / epochs,
)

# 평가

In [ ]:
testGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'test_set'),
    target_size=(100, 100),
)

In [ ]:
pred = model.predict(testGen)

In [ ]:
img_list = testGen.filenames

In [ ]:
pred_list = []
for i in range(len(pred)):
    pred_list.append(np.argmax(pred[i]))

In [ ]:
df = pd.DataFrame(list(zip(img_list, pred_list)), columns =['Name', 'pred'])

In [ ]:
df.to_csv('submission.csv', index=False)

In [ ]:
print("정확도 :", model.evaluate_generator(testGen))[1]